# Демонстрация использования класса `SignalChecker`

Этот ноутбук показывает, как использовать класс `SignalChecker` из `analysis.signal_checker` для проверки наличия определенных наборов сигналов в файлах COMTRADE (`.cfg`).

Демонстрируются:
- Использование стандартной логики проверки сигналов (по умолчанию в классе).
- Использование пользовательской функции для проверки специфического набора сигналов.

In [ ]:
import os
import shutil
import pandas as pd 
import sys
import numpy as np 

# Настройка путей для импорта модулей проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from analysis.signal_checker import SignalChecker 

# Создаем временную директорию для демонстрации
base_sample_dir_sc = "temp_sample_data_signal_checker"
source_dir_sc = os.path.join(base_sample_dir_sc, "source_oscillograms_sc")

if os.path.exists(base_sample_dir_sc):
    shutil.rmtree(base_sample_dir_sc)
os.makedirs(source_dir_sc)

# Функция для создания CFG (упрощенная)
def create_checker_cfg(path, name, content_lines):
    with open(os.path.join(path, f"{name}.cfg"), "w", encoding="utf-8") as f:
        for line in content_lines:
            f.write(line + "\n")

# Файл 1: Должен пройти стандартную проверку (_default_signal_check_logic)
cfg1_content_valid = [
    "Station,Dev,2013", "11,10A,1D", 
    "1,U | BusBar-1 | phase: A,A,,V,1,0,0,0,0,1,1,P", 
    "2,U | BusBar-1 | phase: B,B,,V,1,0,0,0,0,1,1,P", 
    "3,U | BusBar-1 | phase: C,C,,V,1,0,0,0,0,1,1,P",
    "4,U | CableLine-2 | phase: A,A,,V,1,0,0,0,0,1,1,P",
    "5,U | CableLine-2 | phase: B,B,,V,1,0,0,0,0,1,1,P",
    "6,U | CableLine-2 | phase: C,C,,V,1,0,0,0,0,1,1,P",
    "7,I | Bus-1 | phase: A,A,,A,1,0,0,0,0,1,1,P",
    "8,I | Bus-1 | phase: C,C,,A,1,0,0,0,0,1,1,P",
    "9,I | Bus-2 | phase: A,A,,A,1,0,0,0,0,1,1,P",
    "10,I | Bus-2 | phase: C,C,,A,1,0,0,0,0,1,1,P",
    "11,PDR | Bus-1 | phase: PS,,,1",
    "50.0","1","1000,100","01/01/2023,00:00:00.000","01/01/2023,00:00:01.000","ASCII","1.0"
]
create_checker_cfg(source_dir_sc, "osc_sc_valid.cfg", cfg1_content_valid)

# Файл 2: Не пройдет стандартную проверку (отсутствует PDR)
cfg2_content_invalid = [
    "Station,Dev,2013", "10,10A,0D",
    "1,U | BusBar-1 | phase: A,A,,V,1,0,0,0,0,1,1,P", "2,U | BusBar-1 | phase: B,B,,V,1,0,0,0,0,1,1,P", "3,U | BusBar-1 | phase: C,C,,V,1,0,0,0,0,1,1,P",
    "4,U | BusBar-2 | phase: A,A,,V,1,0,0,0,0,1,1,P", "5,U | BusBar-2 | phase: B,B,,V,1,0,0,0,0,1,1,P", "6,U | BusBar-2 | phase: C,C,,V,1,0,0,0,0,1,1,P",
    "7,I | Bus-1 | phase: A,A,,A,1,0,0,0,0,1,1,P", "8,I | Bus-1 | phase: C,C,,A,1,0,0,0,0,1,1,P",
    "9,I | Bus-2 | phase: A,A,,A,1,0,0,0,0,1,1,P", "10,I | Bus-2 | phase: C,C,,A,1,0,0,0,0,1,1,P",
    "50.0","1","1000,100","01/01/2023,00:00:00.000","01/01/2023,00:00:01.000","ASCII","1.0"
]
create_checker_cfg(source_dir_sc, "osc_sc_invalid_no_pdr.cfg", cfg2_content_invalid)

# Файл 3: Для пользовательской проверки (наличие U и I на Bus-2)
cfg3_content_custom = [
    "Station,Dev,2013", "4,4A,0D",
    "1,U | Bus-1 | phase: A,A,,V,1,0,0,0,0,1,1,P", 
    "2,I | Bus-1 | phase: A,A,,A,1,0,0,0,0,1,1,P", 
    "3,U | Bus-2 | phase: B,B,,V,1,0,0,0,0,1,1,P", 
    "4,I | Bus-2 | phase: C,C,,A,1,0,0,0,0,1,1,P", 
    "50.0","1","1000,100","01/01/2023,00:00:00.000","01/01/2023,00:00:01.000","ASCII","1.0"
]
create_checker_cfg(source_dir_sc, "osc_sc_custom_check_target.cfg", cfg3_content_custom)

print(f"Создана директория с примерами: {source_dir_sc}")

## 1. Использование `SignalChecker` со стандартной логикой проверки
Стандартная логика (`_default_signal_check_logic`) проверяет наличие определенного набора напряжений, токов и сигналов PDR.

In [ ]:
checker_default = SignalChecker() 
default_report_path = os.path.join(base_sample_dir_sc, "default_signal_check_report.csv")

print("\nПроверка сигналов со стандартной логикой:")
checker_default.check_signals_in_directory(
    source_dir=source_dir_sc,
    output_csv_path=default_report_path,
    is_print_message=True
)

if os.path.exists(default_report_path):
    print(f"\nСодержимое отчета ({default_report_path}):")
    try: 
        from IPython.display import display
        display(pd.read_csv(default_report_path))
    except ImportError:
        print(pd.read_csv(default_report_path))


## 2. Использование `SignalChecker` с пользовательской логикой проверки
Можно передать свою функцию для проверки наличия сигналов.

In [ ]:
def custom_check_bus2_U_and_I(parsed_signals: list[dict]) -> bool:
    has_U_bus2 = False
    has_I_bus2 = False
    for signal in parsed_signals:
        loc_type = signal.get('location_type')
        sec_num = signal.get('section_number')
        sig_type = signal.get('signal_type')

        if loc_type == 'Bus' and sec_num == '2':
            if sig_type == 'U':
                has_U_bus2 = True
            elif sig_type == 'I':
                has_I_bus2 = True
        if has_U_bus2 and has_I_bus2:
            return True
    return False

custom_report_path = os.path.join(base_sample_dir_sc, "custom_signal_check_report.csv")

print("\nПроверка сигналов с пользовательской логикой (U и I на Bus-2):")
checker_custom_run = SignalChecker() 
checker_custom_run.check_signals_in_directory(
    source_dir=source_dir_sc,
    output_csv_path=custom_report_path,
    custom_signal_check_logic=custom_check_bus2_U_and_I, 
    is_print_message=True
)
if os.path.exists(custom_report_path):
    print(f"\nСодержимое пользовательского отчета ({custom_report_path}):")
    try: 
        from IPython.display import display
        display(pd.read_csv(custom_report_path))
    except ImportError:
        print(pd.read_csv(custom_report_path))

checker_for_file_check = SignalChecker(custom_signal_check_logic=custom_check_bus2_U_and_I)
custom_target_file = os.path.join(source_dir_sc, "osc_sc_custom_check_target.cfg")
status_custom_target = checker_for_file_check.check_file_signals(custom_target_file, is_print_message=True)
print(f"\nСтатус для {os.path.basename(custom_target_file)} (custom logic): {status_custom_target}")

status_default_target_custom_check = checker_for_file_check.check_file_signals(os.path.join(source_dir_sc, "osc_sc_valid.cfg"), is_print_message=True)
print(f"Статус для osc_sc_valid.cfg (custom logic): {status_default_target_custom_check}")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(base_sample_dir_sc):
        shutil.rmtree(base_sample_dir_sc)
        print(f"\nВременная директория {base_sample_dir_sc} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {base_sample_dir_sc}: {e}")